In [ ]:
"""
Update Parameters Here
"""
COLLECTION_NAME = "Quaks"
CONTRACT = "0x07bbdaf30e89ea3ecf6cadc80d6e7c4b0843c729"
BEFORE_TIME = "2021-09-02T00:00:00"  # One day after the last mint (e.g. https://etherscan.io/tx/0x206c846d0d1739faa9835e16ff419d15708a558357a9413619e65dacf095ac7a)
TIME_DELTA = 5

# these should usually stay the same
METHOD = "raritytools"
MINT_ADDRESS = "0x0000000000000000000000000000000000000000"
SLEEP = 1
LIMIT = 200
MAX_OFFSET = 10000
OPENSEA_API_KEY = "YOUR_API_KEY"

In [ ]:
"""
Created on Tue Sep 14 20:17:07 2021
mint data. Doesn't work when Opensea's API is being shitty
@author: nbax1, slight modifications by mdigi14
"""

import time
import requests
import pandas as pd
import datetime

"""
Helper Functions
"""


def getOpenseaEvents(contract, offset=0, occurred_before=BEFORE_TIME):
    print(offset)
    url = "https://api.opensea.io/api/v1/events"

    querystring = {
        "asset_contract_address": contract,
        "only_opensea": "false",
        "offset": str(offset),
        "limit": LIMIT,
        "event_type": "transfer",
        "occurred_before": occurred_before,
    }

    headers = {"Accept": "application/json", "X-API-KEY": OPENSEA_API_KEY}

    response = requests.request("GET", url, headers=headers, params=querystring)
    if response.status_code == 200:
        return response.json()
    else:
        print("error, sleeping then calling again")
        time.sleep(SLEEP * 5)
        return getOpenseaEvents(contract, offset=offset)


def getMintEvents(contract, rarity_db):
    offset = 0
    data = getOpenseaEvents(contract, offset=offset)
    events = data["asset_events"]
    mint_list = []
    before_time = BEFORE_TIME

    while len(data["asset_events"]) >= LIMIT and offset <= MAX_OFFSET:
        if offset >= MAX_OFFSET:
            old_before_time = events[-1]["transaction"]["timestamp"]
            datetime_before_time = datetime.datetime.strptime(
                old_before_time, "%Y-%m-%dT%H:%M:%S"
            )
            time_change = datetime.timedelta(minutes=TIME_DELTA)

            new_time = datetime_before_time + time_change
            before_time = new_time.strftime("%Y-%m-%dT%H:%M:%S")
            print("resetting offset from ", old_before_time, " to ", new_time)
            offset = 0
        else:
            offset = offset + LIMIT
        time.sleep(SLEEP)
        data = getOpenseaEvents(contract, offset=offset, occurred_before=before_time)

        for event in data["asset_events"]:
            if event["transaction"] is not None:
                events.append(event)
            else:
                print("fail tx")

    index = 0
    seen = set()
    for event in events:
        if event["event_type"] == "transfer":  # check if its a transfer
            if event["from_account"]["address"] == MINT_ADDRESS:
                tokenId = int(event["asset"]["token_id"])
                if tokenId not in rarity_db["TOKEN_ID"].values:
                    print("missing rarity for ", tokenId)
                    continue

                if event["transaction"] is None:
                    mint_dict = {
                        "txid": "error",
                        "to_account": event["to_account"]["address"],
                        "TOKEN_ID": tokenId,
                        "current_owner": event["asset"]["owner"]["address"],
                        "rank": str(
                            int(rarity_db[rarity_db["TOKEN_ID"] == tokenId]["Rank"])
                        ),
                        "time": "error",
                    }
                    print("fail tx")
                else:
                    mint_dict = {
                        "txid": event["transaction"]["transaction_hash"],
                        "to_account": event["to_account"]["address"],
                        "TOKEN_ID": tokenId,
                        "current_owner": event["asset"]["owner"]["address"],
                        "rank": str(
                            int(rarity_db[rarity_db["TOKEN_ID"] == tokenId]["Rank"])
                        ),
                        "time": event["transaction"]["timestamp"],
                    }

                if mint_dict["TOKEN_ID"] not in seen:
                    mint_list.append(mint_dict)
                    seen.add(mint_dict["TOKEN_ID"])
    return pd.DataFrame.from_dict(mint_list)

In [ ]:
"""
Gerenerate Datast
"""

RARITY_CSV = "../metadata/rarity_data/{}_{}.csv".format(COLLECTION_NAME, METHOD)
RARITY_DB = pd.read_csv(RARITY_CSV)

mintDb = getMintEvents(CONTRACT, RARITY_DB)
mintDb = mintDb.sort_values(by=["TOKEN_ID"])
mintDb.to_csv("../minting_data/{}_minting.csv".format(COLLECTION_NAME))